In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import box

from src.stop_explorer import StopExplorer
from src.grid_partitioning import Grid

### Main code

In [2]:
# Create a GeoDataFrame for the stops dataset.
path_stops = './data_simulator/huge_dataset/dataset_simulator_trajectories.compressed.parquet.stops.parquet'
stop_explorer = StopExplorer(path_stops)

df_stops = stop_explorer.get_df_stops()
display(df_stops)
df_stops.info()

,datetime,uid,leaving_datetime,duration_secs,geometry,hour_start,hour_end,weekday
0,2019-07-01 00:00:00,0,2019-07-01 13:52:00,49920.0,POINT (-84.36998 33.75245),0,13,0
1,2019-07-01 13:52:00,0,2019-07-01 15:48:00,6960.0,POINT (-84.37004 33.75206),13,15,0
2,2019-07-01 15:52:00,0,2019-07-01 16:58:00,3960.0,POINT (-84.36825 33.75266),15,16,0
3,2019-07-01 16:58:00,0,2019-07-01 18:04:00,3960.0,POINT (-84.36998 33.75245),16,18,0
4,2019-07-01 18:06:00,0,2019-07-01 19:28:00,4920.0,POINT (-84.36998 33.75245),18,19,0
...,...,...,...,...,...,...,...,...
4393268,2019-07-09 20:58:00,99999,2019-07-09 22:54:00,6960.0,POINT (-84.37187 33.75968),20,22,1
4393269,2019-07-09 22:54:00,99999,2019-07-10 07:52:00,32280.0,POINT (-84.36952 33.7616),22,7,1
4393270,2019-07-10 07:52:00,99999,2019-07-10 16:36:00,31440.0,POINT (-84.38731 33.76169),7,16,2
4393271,2019-07-10 16:36:00,99999,2019-07-10 22:44:00,22080.0,POINT (-84.36952 33.7616),16,22,2


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4393273 entries, 0 to 4393272
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   datetime          datetime64[ns]
 1   uid               int64         
 2   leaving_datetime  datetime64[ns]
 3   duration_secs     float64       
 4   geometry          geometry      
 5   hour_start        uint8         
 6   hour_end          uint8         
 7   weekday           uint8         
dtypes: datetime64[ns](2), float64(1), geometry(1), int64(1), uint8(3)
memory usage: 180.2 MB


### Plot some stop segments' statistics

In [3]:
# Show the frequency of the top-20 pairs (hour_start, hour_stop)
prova = stop_explorer.get_stops_temporal_intervals_freqs().head(20)
display(prova)

hour_start  hour_end
7           16          0.052407
16          18          0.037260
17          19          0.032959
8           16          0.032063
18          20          0.031738
19          21          0.029515
20          22          0.028800
7           15          0.027345
17          18          0.026410
16          17          0.025518
8           17          0.025403
21          22          0.025382
22          23          0.025040
15          17          0.023631
6           15          0.023025
18          19          0.022362
0           8           0.021901
21          23          0.021509
19          20          0.021099
20          21          0.020831
Name: proportion, dtype: float64

### Visualize the grid and where the stops fall in it on a map

In [4]:
grid = Grid(grid_cell_length_meters = 100)
grid.compute_grid_for_geodata(df_stops)

mappa = grid.generate_grid_map()
mappa